## 10월 26일 목요일 연습문제

- 오늘 실습해서 만든 코사인유사도 행렬 [0]번 영화 토이스토리와 가장 유사한(값이 큰) 영화 제목을 출력해주세요.
- 장르가 중복되는 것은 관계 없이 가장 큰 값 10개에 대해 출력해주세요.

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [70]:
data=pd.read_csv('archive/movies_metadata.csv')
data=data.head(5000)
data

C:\Users\user\AppData\Local\Temp\ipykernel_9920\3117426312.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('archive/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,False,NaN,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 27, 'n...",NaN,43715,tt0050294,en,The Deadly Mantis,The calving of an Arctic iceberg releases a gi...,...,1957-05-01,0.0,79.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,This Was the Day That Engulfed the World in Te...,The Deadly Mantis,False,5.3,16.0
4996,False,NaN,60000000,"[{'id': 18, 'name': 'Drama'}]",NaN,10052,tt0259288,en,Dragonfly,A grieving doctor is being contacted by his la...,...,2002-02-22,52322400.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,When someone you love dies... are they gone fo...,Dragonfly,False,6.2,209.0
4997,False,"{'id': 217704, 'name': 'The Vampire Chronicles...",35000000,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",NaN,11979,tt0238546,en,Queen of the Damned,Lestat de Lioncourt is awakened from his slumb...,...,2002-02-10,45479110.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,This time there are no interviews.,Queen of the Damned,False,5.5,247.0
4998,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,75151,tt0260746,en,Big Bad Love,Vietnam veteran Leon Barlow is struggling as a...,...,2001-10-11,0.0,111.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Big Bad Love,False,6.5,4.0


In [71]:
data = data[['id','genres', 'vote_average', 'vote_count','popularity','title', 'overview']]
data

,id,genres,vote_average,vote_count,popularity,title,overview
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,21.946943,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,17.015539,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",6.5,92.0,11.7129,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",6.1,34.0,3.859495,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]",5.7,173.0,8.387519,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...
4995,43715,"[{'id': 53, 'name': 'Thriller'}, {'id': 27, 'n...",5.3,16.0,1.738062,The Deadly Mantis,The calving of an Arctic iceberg releases a gi...
4996,10052,"[{'id': 18, 'name': 'Drama'}]",6.2,209.0,4.924078,Dragonfly,A grieving doctor is being contacted by his la...
4997,11979,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",5.5,247.0,8.436243,Queen of the Damned,Lestat de Lioncourt is awakened from his slumb...
4998,75151,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",6.5,4.0,0.437438,Big Bad Love,Vietnam veteran Leon Barlow is struggling as a...


In [72]:
tmp=data['vote_count'].quantile(0.9)
data=data[data['vote_count']>=tmp]

In [73]:
data

,id,genres,vote_average,vote_count,popularity,title,overview
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,21.946943,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,17.015539,Jumanji,When siblings Judy and Peter discover an encha...
5,949,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",7.7,1886.0,17.924927,Heat,"Obsessive master thief, Neil McCauley leads a ..."
9,710,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",6.6,1194.0,14.686036,GoldenEye,James Bond must unmask the mysterious head of ...
15,524,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",7.8,1343.0,10.137389,Casino,The life of the gambling paradise – Las Vegas ...
...,...,...,...,...,...,...,...
4863,120,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8.0,8892.0,32.070725,The Lord of the Rings: The Fellowship of the Ring,"Young hobbit Frodo Baggins, after inheriting a..."
4865,453,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",7.7,3087.0,11.93646,A Beautiful Mind,"At Princeton University, John Nash struggles t..."
4880,855,"[{'id': 28, 'name': 'Action'}, {'id': 36, 'nam...",7.2,1832.0,10.064446,Black Hawk Down,When U.S. Rangers and an elite Delta Force tea...
4936,10229,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",7.5,1057.0,8.054499,A Walk to Remember,"When the popular, restless Landon Carter is fo..."


In [74]:
data['genres']=data['genres'].apply(eval)
data['genres']

C:\Users\user\AppData\Local\Temp\ipykernel_9920\2079851878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres']=data['genres'].apply(eval)


0       [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1       [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
5       [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
9       [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...
15      [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...
                              ...                        
4863    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
4865    [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...
4880    [{'id': 28, 'name': 'Action'}, {'id': 36, 'nam...
4936    [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...
4977    [{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...
Name: genres, Length: 500, dtype: object

In [75]:
data['genres']=data['genres'].apply(lambda x: " ".join([i['name'] for i in x]))
data['genres']

C:\Users\user\AppData\Local\Temp\ipykernel_9920\4035604079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres']=data['genres'].apply(lambda x: " ".join([i['name'] for i in x]))


0           Animation Comedy Family
1          Adventure Fantasy Family
5       Action Crime Drama Thriller
9         Adventure Action Thriller
15                      Drama Crime
                   ...             
4863       Adventure Fantasy Action
4865                  Drama Romance
4880             Action History War
4936                  Drama Romance
4977           Drama Thriller Crime
Name: genres, Length: 500, dtype: object

In [76]:
data

,id,genres,vote_average,vote_count,popularity,title,overview
0,862,Animation Comedy Family,7.7,5415.0,21.946943,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Adventure Fantasy Family,6.9,2413.0,17.015539,Jumanji,When siblings Judy and Peter discover an encha...
5,949,Action Crime Drama Thriller,7.7,1886.0,17.924927,Heat,"Obsessive master thief, Neil McCauley leads a ..."
9,710,Adventure Action Thriller,6.6,1194.0,14.686036,GoldenEye,James Bond must unmask the mysterious head of ...
15,524,Drama Crime,7.8,1343.0,10.137389,Casino,The life of the gambling paradise – Las Vegas ...
...,...,...,...,...,...,...,...
4863,120,Adventure Fantasy Action,8.0,8892.0,32.070725,The Lord of the Rings: The Fellowship of the Ring,"Young hobbit Frodo Baggins, after inheriting a..."
4865,453,Drama Romance,7.7,3087.0,11.93646,A Beautiful Mind,"At Princeton University, John Nash struggles t..."
4880,855,Action History War,7.2,1832.0,10.064446,Black Hawk Down,When U.S. Rangers and an elite Delta Force tea...
4936,10229,Drama Romance,7.5,1057.0,8.054499,A Walk to Remember,"When the popular, restless Landon Carter is fo..."


In [77]:
tfidf=TfidfVectorizer()

In [78]:
tfidf_mat=tfidf.fit_transform(data['genres'])

In [79]:
tfidf_df=pd.DataFrame(tfidf_mat.toarray(), columns=tfidf.get_feature_names_out())
tfidf_df

,action,adventure,animation,comedy,crime,drama,family,fantasy,fiction,history,horror,music,mystery,romance,science,thriller,war,western
0,0.000000,0.000000,0.684461,0.437024,0.000000,0.000000,0.583544,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,0.000000,0.496438,0.000000,0.000000,0.000000,0.000000,0.606872,0.620689,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
2,0.492218,0.000000,0.000000,0.000000,0.582542,0.419629,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.492218,0.000000,0.0
3,0.573030,0.585896,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.573030,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.811403,0.584487,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.521314,0.533019,0.000000,0.000000,0.000000,0.000000,0.000000,0.666425,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
496,0.000000,0.000000,0.000000,0.000000,0.000000,0.560064,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.828449,0.0,0.000000,0.000000,0.0
497,0.367192,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.667979,0.0,0.0,0.0,0.000000,0.0,0.000000,0.647282,0.0
498,0.000000,0.000000,0.000000,0.000000,0.000000,0.560064,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.828449,0.0,0.000000,0.000000,0.0


In [80]:
cosine_similarity(tfidf_df)

array([[1.        , 0.3541364 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.3541364 , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.18073843, 0.23501929,
        0.8704722 ],
       ...,
       [0.        , 0.        , 0.18073843, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.23501929, ..., 0.        , 1.        ,
        0.26999058],
       [0.        , 0.        , 0.8704722 , ..., 0.        , 0.26999058,
        1.        ]])

In [81]:
cosine_similarity(tfidf_df)[0]

array([1.        , 0.3541364 , 0.        , 0.        , 0.        ,
       0.22443568, 0.        , 0.51965981, 0.22892439, 0.        ,
       0.73997229, 0.        , 0.        , 0.        , 0.4370241 ,
       0.        , 0.        , 0.19780967, 0.        , 0.        ,
       0.56412632, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.4370241 , 0.4370241 , 0.18457278, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.23310648, 0.        ,
       0.22892439, 0.22892439, 0.82251485, 0.17441201, 0.        ,
       0.        , 0.        , 0.        , 0.32310383, 0.        ,
       0.        , 0.        , 0.46026541, 0.63989718, 0.        ,
       0.4370241 , 0.        , 0.22892439, 0.        , 0.7494637 ,
       0.        , 0.        , 0.72904972, 0.        , 0.71968376,
       0.        , 0.        , 0.        , 0.        , 0.7494637 ,
       0.53732606, 0.8994498 , 0.26047833, 0.        , 0.90245

In [82]:
cosine_similarity(tfidf_df)[0].argsort()[-11:]

array([276, 103, 401,  74, 278, 296, 303, 419, 375, 487,   0], dtype=int64)

In [83]:
cosine_similarity(tfidf_df)[0][cosine_similarity(tfidf_df)[0].argsort()[-11:]]

array([0.8994498 , 0.8994498 , 0.90245203, 0.90245203, 0.90245203,
       0.90245203, 0.90245203, 1.        , 1.        , 1.        ,
       1.        ])

In [84]:
data.title.iloc[[487, 375, 419, 303, 296, 278, 74, 401, 103, 276]]

4756                    Monsters, Inc.
2997                       Toy Story 2
3629                       Chicken Run
2242                      A Bug's Life
2181                              Antz
1976    One Hundred and One Dalmatians
608                     The Aristocats
3362             The Road to El Dorado
996             The Sword in the Stone
1971                Lady and the Tramp
Name: title, dtype: object